In [1]:
! pip install datasets transformers seqeval
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
import statistics as stats
from datasets import load_dataset, load_metric

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer

import random 
import numpy as np
import torch

def random_seed(seed_value): 
    np.random.seed(seed_value) 
    torch.manual_seed(seed_value)
    random.seed(seed_value) 


random_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=1295e833991e55bc2a6958354c18adbc45a2f33e96b7b821df0d424f2ea93406
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [2]:
os.environ["WANDB_DISABLED"] = "true"

In [3]:
ncbi = load_dataset("ghadeermobasher/BC5CDR-Chemical-Disease")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

['Selegiline', '-', 'induced', 'postural', 'hypotension', 'in', 'Parkinson', "'", 's', 'disease', ':', 'a', 'longitudinal', 'study', 'on', 'the', 'effects', 'of', 'drug', 'withdrawal', '.']
['OBJECTIVES', ':', 'The', 'United', 'Kingdom', 'Parkinson', "'", 's', 'Disease', 'Research', 'Group', '(', 'UKPDRG', ')', 'trial', 'found', 'an', 'increased', 'mortality', 'in', 'patients', 'with', 'Parkinson', "'", 's', 'disease', '(', 'PD', ')', 'randomized', 'to', 'receive', '10', 'mg', 'selegiline', 'per', 'day', 'and', 'L', '-', 'dopa', 'compared', 'with', 'those', 'taking', 'L', '-', 'dopa', 'alone', '.']
['Recently', ',', 'we', 'found', 'that', 'therapy', 'with', 'selegiline', 'and', 'L', '-', 'dopa', 'was', 'associated', 'with', 'selective', 'systolic', 'orthostatic', 'hypotension', 'which', 'was', 'abolished', 'by', 'withdrawal', 'of', 'selegiline', '.']
['This', 'unwanted', 'effect', 'on', 'postural', 'blood', 'pressure', 'was', 'not', 'the', 'result', 'of', 'underlying', 'autonomic', 'fa

Generating validation split: 0 examples [00:00, ? examples/s]

['22', '-', 'oxacalcitriol', 'suppresses', 'secondary', 'hyperparathyroidism', 'without', 'inducing', 'low', 'bone', 'turnover', 'in', 'dogs', 'with', 'renal', 'failure', '.']
['BACKGROUND', ':', 'Calcitriol', 'therapy', 'suppresses', 'serum', 'levels', 'of', 'parathyroid', 'hormone', '(', 'PTH', ')', 'in', 'patients', 'with', 'renal', 'failure', 'but', 'has', 'several', 'drawbacks', ',', 'including', 'hypercalcemia', 'and', '/', 'or', 'marked', 'suppression', 'of', 'bone', 'turnover', ',', 'which', 'may', 'lead', 'to', 'adynamic', 'bone', 'disease', '.']
['A', 'new', 'vitamin', 'D', 'analogue', ',', '22', '-', 'oxacalcitriol', '(', 'OCT', ')', ',', 'has', 'been', 'shown', 'to', 'have', 'promising', 'characteristics', '.']
['This', 'study', 'was', 'undertaken', 'to', 'determine', 'the', 'effects', 'of', 'OCT', 'on', 'serum', 'PTH', 'levels', 'and', 'bone', 'turnover', 'in', 'states', 'of', 'normal', 'or', 'impaired', 'renal', 'function', '.']
['METHODS', ':', 'Sixty', 'dogs', 'were', '

Generating test split: 0 examples [00:00, ? examples/s]

['Torsade', 'de', 'pointes', 'ventricular', 'tachycardia', 'during', 'low', 'dose', 'intermittent', 'dobutamine', 'treatment', 'in', 'a', 'patient', 'with', 'dilated', 'cardiomyopathy', 'and', 'congestive', 'heart', 'failure', '.']
['The', 'authors', 'describe', 'the', 'case', 'of', 'a', '56', '-', 'year', '-', 'old', 'woman', 'with', 'chronic', ',', 'severe', 'heart', 'failure', 'secondary', 'to', 'dilated', 'cardiomyopathy', 'and', 'absence', 'of', 'significant', 'ventricular', 'arrhythmias', 'who', 'developed', 'QT', 'prolongation', 'and', 'torsade', 'de', 'pointes', 'ventricular', 'tachycardia', 'during', 'one', 'cycle', 'of', 'intermittent', 'low', 'dose', '(', '2', '.', '5', 'mcg', '/', 'kg', 'per', 'min', ')', 'dobutamine', '.']
['This', 'report', 'of', 'torsade', 'de', 'pointes', 'ventricular', 'tachycardia', 'during', 'intermittent', 'dobutamine', 'supports', 'the', 'hypothesis', 'that', 'unpredictable', 'fatal', 'arrhythmias', 'may', 'occur', 'even', 'with', 'low', 'doses', '

  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
ncbi["train"][0]

{'id': '0',
 'tokens': ['Selegiline',
  '-',
  'induced',
  'postural',
  'hypotension',
  'in',
  'Parkinson',
  "'",
  's',
  'disease',
  ':',
  'a',
  'longitudinal',
  'study',
  'on',
  'the',
  'effects',
  'of',
  'drug',
  'withdrawal',
  '.'],
 'ner_tags': [3, 0, 0, 1, 2, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [5]:
label_list = ncbi["train"].features[f"ner_tags"].feature.names
print(len(label_list))
label_list

5


['O', 'B-Disease', 'I-Disease', 'B-Chemical', 'I-Chemical']

In [6]:
MODEL_NAME = "AnonymousSub/fpdm_models_scibert_hybrid_epochs_4"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/427 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
label_all_tokens = True

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [10]:
tokenize_and_align_labels(ncbi['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[102, 6998, 4980, 16863, 30107, 579, 2651, 22635, 21983, 121, 12379, 2505, 112, 1288, 862, 106, 7051, 527, 191, 111, 1056, 131, 1698, 12751, 205, 103], [102, 6454, 862, 111, 4274, 11967, 12379, 2505, 112, 1288, 849, 583, 145, 4175, 4008, 17640, 546, 3303, 797, 130, 1175, 3380, 121, 568, 190, 12379, 2505, 112, 1288, 145, 2841, 546, 5460, 147, 5380, 566, 1529, 6998, 4980, 16863, 30107, 309, 2181, 137, 152, 579, 4687, 30110, 1031, 190, 1052, 3841, 152, 579, 4687, 30110, 3957, 205, 103], [102, 2803, 422, 185, 797, 198, 2223, 190, 6998, 4980, 16863, 30107, 137, 152, 579, 4687, 30110, 241, 1111, 190, 5704, 11152, 3816, 14657, 21983, 334, 241, 17927, 214, 12751, 131, 6998, 4980, 16863, 30107, 205, 103], [102, 238, 20628, 907, 191, 22635, 1702, 2241, 241, 302, 111, 1186, 131, 3954, 16035, 3018, 205, 103], [102, 111, 7415, 131, 238, 527, 267, 147, 4485, 580, 1061, 2116, 121, 106, 4396, 5055, 131, 568, 137, 147, 2100, 111, 532, 3092, 131, 111, 5283, 6091, 131, 16018, 6998, 4980, 1

In [11]:
tokenized_datasets = ncbi.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [12]:
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(label_list))

Downloading:   0%|          | 0.00/706 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at AnonymousSub/fpdm_models_scibert_hybrid_epochs_4 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
args = TrainingArguments("SCIBERT-VOCAB-UNCASED-finetuned-NER",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=4,
    weight_decay=0.01
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [15]:
metric = load_metric("seqeval")

In [16]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4561
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 572


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.102700,0.877135,0.905957,0.891313,0.965878
2,No log,0.102628,0.887933,0.913133,0.900356,0.967639
3,No log,0.106361,0.893068,0.921064,0.906850,0.968941
4,0.097400,0.110353,0.901557,0.915615,0.908531,0.969626


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4582
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4582
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***

TrainOutput(global_step=572, training_loss=0.08869743597257387, metrics={'train_runtime': 238.4607, 'train_samples_per_second': 76.507, 'train_steps_per_second': 2.399, 'total_flos': 760148192502660.0, 'train_loss': 0.08869743597257387, 'epoch': 4.0})

In [19]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4582
  Batch size = 64


{'eval_loss': 0.11035347729921341,
 'eval_precision': 0.9015566062795516,
 'eval_recall': 0.9156145462393439,
 'eval_f1': 0.9085311989720802,
 'eval_accuracy': 0.9696262561631978,
 'eval_runtime': 18.9553,
 'eval_samples_per_second': 241.726,
 'eval_steps_per_second': 3.798,
 'epoch': 4.0}

In [20]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 4798
  Batch size = 64


{'Chemical': {'precision': 0.9368964368964369,
  'recall': 0.9401837928153718,
  'f1': 0.9385372362605288,
  'number': 11970},
 'Disease': {'precision': 0.7994229976645143,
  'recall': 0.8567432273262662,
  'f1': 0.8270911804420439,
  'number': 6792},
 'overall_precision': 0.8850241045046913,
 'overall_recall': 0.9099776143268308,
 'overall_f1': 0.8973274117677975,
 'overall_accuracy': 0.9672872283709723}